# Subtitle Collection

## Step #1: Collect Youtube Video ID list for Children's works

- The Youtube API now has a madeforkids flag
   - https://developers.google.com/youtube/v3/docs/videos
- VideoCategories can be listed at https://developers.google.com/youtube/v3/docs/videoCategories/list?apix_params=%7B%22part%22%3A%5B%22snippet%22%5D%2C%22regionCode%22%3A%22us%22%7D (part:'snippet' ensures that the name is included)




In [ ]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

# Disable OAuthlib's HTTPS verification when running locally.
# *DO NOT* leave this option enabled in production.
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "../client_secret_312872897581-qii8av1f39ko8iavk4qp3sm8lqnp74op.apps.googleusercontent.com.json"

# Get credentials and create an API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

In [67]:
categories = youtube.videoCategories().list(part="snippet",regionCode="US")
category_list = categories.execute()
print("\n".join(["{}\t{}".format(i['id'], i['snippet']['title']) for i in category_list['items']]))

1	Film & Animation
2	Autos & Vehicles
10	Music
15	Pets & Animals
17	Sports
18	Short Movies
19	Travel & Events
20	Gaming
21	Videoblogging
22	People & Blogs
23	Comedy
24	Entertainment
25	News & Politics
26	Howto & Style
27	Education
28	Science & Technology
29	Nonprofits & Activism
30	Movies
31	Anime/Animation
32	Action/Adventure
33	Classics
34	Comedy
35	Documentary
36	Drama
37	Family
38	Foreign
39	Horror
40	Sci-Fi/Fantasy
41	Thriller
42	Shorts
43	Shows
44	Trailers


In [63]:
request = youtube.videos().list(
    part="snippet,status",
    id= "jil0WCh_UoQ",
    maxResults=25
)
response = request.execute()

try:
    for item in response['items']:
        snippet, status = item['snippet'], item['status']
        title = snippet["title"] if "title" in snippet else None
        madeForKids = status["madeForKids"] if "madeForKids" in status else None
        tags = snippet["tags"] if "tags" in snippet else None
        category = snippet["categoryId"] if "categoryId" in snippet else None
        print(title, madeForKids, tags, category)
        #print(json.dumps(item['snippet'], indent=4))
except:
    raise
    print(json.dumps(response, indent=4))

Peppa Pig and Suzie Sheep Whistle False ['Tumblr', 'Vimeo', 'Peppa Pig', 'British TV', 'British Television', 'Cartoons', 'Child Cartoons', 'Kid Cartoons', 'Toddler Cartoons', 'Suzie Sheep', 'Whistle', 'Whistling', 'Funny', 'Hangs up', 'Upset', 'Learning to Whistle', 'Learns'] 23


In [84]:
import json
request = youtube.search().list(
    part="snippet",
    q="children dinosaurs",
    safeSearch="strict",
    maxResults=25,
    type="video",
    regionCode="US",
    relevanceLanguage="en",
    #videoCategoryId=23
)
response = request.execute()

print(json.dumps(response, indent=4))

{
    "kind": "youtube#searchListResponse",
    "etag": "OrFUQ6E9eGmUCGwQSNSsBQRRxSw",
    "nextPageToken": "CBkQAA",
    "regionCode": "US",
    "pageInfo": {
        "totalResults": 1000000,
        "resultsPerPage": 25
    },
    "items": [
        {
            "kind": "youtube#searchResult",
            "etag": "RVBf8HmS9XPwXJOyZpswrbYD5oI",
            "id": {
                "kind": "youtube#video",
                "videoId": "A7Mt-WF_y_Q"
            },
            "snippet": {
                "publishedAt": "2017-05-15T12:52:21Z",
                "channelId": "UC5Ti4_DVp7LW34PjEwB13Xg",
                "title": "Morphle Dinosaurs for kids! Super hero Morphle",
                "description": "Morphle Playlist: https://www.youtube.com/playlist?list=PLmwpvieHKxgF2A7612PdKabO1EEjfe5aP Morphle Playlist: ...",
                "thumbnails": {
                    "default": {
                        "url": "https://i.ytimg.com/vi/A7Mt-WF_y_Q/default.jpg",
                        "widt

In [85]:
video_ids = [i['id']['videoId'] for i in response['items']]
print(video_ids)

['A7Mt-WF_y_Q', 'RZcm-glKy7Y', 'Mtc-80FJ5aQ', '0vW4_T-Wf6w', 'phNab4hX9AY', 'qT8qx_SfkI4', 'uo5So-Y8A4o', 'Ge54aBlfX-o', 'dktnOPfE7Dc', 'u1sYaKlFrWc', 'N0kyy9CD1OQ', 'YiztfQQF_jY', '-_Bj7Nila3o', 'dp7bMGRgdjk', 'j3y8ut-EphY', 'iORN3ywg3ik', 'dby1qndJnGs', 'zBkjFGYKMhA', 'G3gXWDYpLAE', 'YejdDnEXGqM', 'aRbXkjrdsAg', 'ME8SkJWfkLY', '0nxZJ4qzyEU', 'l_wisuBlhyU', 'YQX7JYQjDOQ']


## Step #2: Download Transcripts

In [3]:
list(YouTubeTranscriptApi.list_transcripts(test_id))

In [5]:
transcript = YouTubeTranscriptApi.get_transcript(test_id)
transcript.language

AttributeError: 'list' object has no attribute 'language'

In [81]:
test_id = "jil0WCh_UoQ" # https://www.youtube.com/watch?v=jil0WCh_UoQ
from youtube_transcript_api import YouTubeTranscriptApi

def get_transcript(vid_id):
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(vid_id)
        transcript = transcript_list.find_transcript(['en'])
        vid_id, is_generated, transcript = transcript.video_id, transcript.is_generated, transcript.fetch()
        return (is_generated, transcript)
    except:
        return None, None

get_transcript(test_id)

(True,
 [{'text': "mmm-hmm what's wrong pepper I can't",
   'start': 2.33,
   'duration': 6.82},
  {'text': 'whistle but everybody else can never',
   'start': 5.85,
   'duration': 5.1},
  {'text': "mind I'm making cookies", 'start': 9.15, 'duration': 3.93},
  {'text': 'would you like to lick the spoon no',
   'start': 10.95,
   'duration': 5.76},
  {'text': 'thank you mommy can I ring Suzy sheep',
   'start': 13.08,
   'duration': 7.609},
  {'text': 'instead okay pepper', 'start': 16.71, 'duration': 3.979},
  {'text': 'hello mrs. Pig hello mrs. sheep can',
   'start': 22.97,
   'duration': 7.14},
  {'text': 'pepper talk to Susie please hello Susie',
   'start': 26.13,
   'duration': 5.699},
  {'text': 'hello pepper', 'start': 30.11, 'duration': 4.21},
  {'text': "what are you doing I'm learning to",
   'start': 31.829,
   'duration': 6.121},
  {'text': "whistle but I can't do it yet hmm that",
   'start': 34.32,
   'duration': 4.95},
  {'text': 'sounds hard', 'start': 37.95, 'duration

In [86]:
transcripts = []
for vid_id in video_ids:
    is_generated, transcript = get_transcript(vid_id)
    transcripts.append((vid_id, transcript))

In [87]:
transcripts

[('A7Mt-WF_y_Q',
  [{'text': 'I wonder how the toy dinosaurs are doing',
    'start': 0.0,
    'duration': 27.26},
   {'text': 'mournful into a pterodactyl', 'start': 20.88, 'duration': 6.38},
   {'text': "great you are here to save me I'm stuck",
    'start': 39.07,
    'duration': 7.65},
   {'text': "on this island and it's filled with",
    'start': 42.47,
    'duration': 4.25},
   {'text': "giant monsters monsters there's a toy",
    'start': 46.78,
    'duration': 14.29},
   {'text': 'sinusoid no wait it will lead you there',
    'start': 54.67,
    'duration': 8.11},
   {'text': 'not scary', 'start': 61.07, 'duration': 6.24},
   {'text': "they are my toys professor Rasheed's",
    'start': 62.78,
    'duration': 6.66},
   {'text': 'light reed bed to light', 'start': 67.31, 'duration': 6.44},
   {'text': '[Music]', 'start': 69.44, 'duration': 11.71},
   {'text': 'wow really into a tier X 2', 'start': 73.75, 'duration': 10.92},
   {'text': '[Music]', 'start': 81.15, 'duration': 7.2

## Step 3: Parse Dataset

Ideas - if there's a pause between lines, treat it as a new sentence or paragraph.